In [1]:
# Dependencies
import os 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress 

#Reading in Files
cdc_df = pd.read_csv("Resource/VSRR_Provisional_Drug_Overdose_Death_Counts_2015_2021.csv")
care_df = pd.read_csv("Resource/Medicare_Opioid_Prescribing_Data_2013_2019.csv", low_memory = False)
caid_df = pd.read_csv("Resource/Medicaid_Opioid_Prescribing_data_2013_2019.csv", low_memory = False)

In [8]:
# Filtering Medcaid dropping year 2013 & 2014 and filtering for all plan types
caid_year = caid_df.loc[(caid_df["Year"] != 2013) & (caid_df["Year"] != 2014)]

caid_clean = caid_year.loc[caid_year["Plan_Type"]=="All"]

In [10]:
# Changing column name to state for merging later
caid_clean = caid_clean.rename(columns={"Geo_Desc":"State"})

caid_clean

Year   Geo_Lvl  Geo_Cd          State Plan_Type  Tot_Opioid_Clms  \
0    2019  National     NaN       National       All       21978286.0   
3    2019     State     1.0        Alabama       All         224310.0   
6    2019     State     2.0         Alaska       All          65394.0   
9    2019     State     4.0        Arizona       All         595206.0   
12   2019     State     5.0       Arkansas       All         216081.0   
..    ...       ...     ...            ...       ...              ...   
765  2015     State    51.0       Virginia       All         583116.0   
768  2015     State    53.0     Washington       All        1136474.0   
771  2015     State    54.0  West Virginia       All         520370.0   
774  2015     State    55.0      Wisconsin       All         962286.0   
777  2015     State    56.0        Wyoming       All          39088.0   

        Tot_Clms  Opioid_Prscrbng_Rate  Opioid_Prscrbng_Rate_5Y_Chg  \
0    677248025.0                  3.25                        -3.22   
3      7230251.0                  3.10                        -3.23   
6      1434139.0                  4.56                        -3.33   
9     14923326.0                  3.99                        -5.14   
12     4999626.0                  4.32                        -3.12   
..           ...                   ...                          ...   
765   10186332.0                  5.72                          NaN   
768   14016747.0                  8.11                          NaN   
771    9293729.0                  5.60                          NaN   
774   11306189.0                  8.51                          NaN   
777     534212.0                  7.32                          NaN   

     Opioid_Prscrbng_Rate_1Y_Chg  LA_Tot_Opioid_Clms  LA_Opioid_Prscrbng_Rate  \
0                          -0.44           3108845.0                    14.15   
3                          -1.08             10308.0                     4.60   
6                          -1.20              8268.0                    12.64   
9                          -0.75             44311.0                     7.44   
12                         -0.88              8129.0                     3.76   
..                           ...                 ...                      ...   
765                        -0.62             42311.0                     7.26   
768                        -0.89            106143.0                     9.34   
771                        -1.30             25575.0                     4.91   
774                        -0.35            146248.0                    15.20   
777                        -0.48              3792.0                     9.70   

     LA_Opioid_Prscrbng_Rate_5Y_Chg  LA_Opioid_Prscrbng_Rate_1Y_Chg  
0                              6.67                            4.62  
3                              0.16                            0.48  
6                              0.88                           -0.71  
9                             -2.79                           -0.52  
12                            -0.34                           -1.05  
..                              ...                             ...  
765                             NaN                           -0.26  
768                             NaN                            0.18  
771                             NaN                            0.66  
774                             NaN                            1.04  
777                             NaN                           -0.49  

[260 rows x 14 columns]

In [7]:
# Filtering Medicaid Data by Breakout Type and Perscription Geographic Levels for National or State levels
care_year = care_df.loc[(care_df["Year"] != 2013) & (care_df["Year"] != 2014)]

care_clean = care_year.loc[(care_year["Breakout_Type"]=="Totals") & (care_year["Prscrbr_Geo_Lvl"]=="National") | (care_year["Prscrbr_Geo_Lvl"]=="State")]

In [8]:
# Changing column name to state for merging late
care_clean = care_clean.rename(columns={"Prscrbr_Geo_Desc":"State"})

care_clean

Year Prscrbr_Geo_Lvl  Prscrbr_Geo_Cd                     State  \
0       2019        National             NaN                  National   
3       2019           State             1.0                   Alabama   
4       2019           State             2.0                    Alaska   
5       2019           State             4.0                   Arizona   
6       2019           State             5.0                  Arkansas   
...      ...             ...             ...                       ...   
120121  2015           State            60.0            American Samoa   
120122  2015           State            66.0                      Guam   
120123  2015           State            69.0  Northern Mariana Islands   
120124  2015           State            72.0               Puerto Rico   
120125  2015           State            78.0            Virgin Islands   

        RUCA_Cd Breakout_Type Breakout  Tot_Prscrbrs  Tot_Opioid_Prscrbrs  \
0           NaN        Totals  Overall     1239380.0             803549.0   
3           NaN        Totals  Overall       15575.0               9739.0   
4           NaN        Totals  Overall        2847.0               1935.0   
5           NaN        Totals  Overall       25876.0              16578.0   
6           NaN        Totals  Overall        9700.0               7258.0   
...         ...           ...      ...           ...                  ...   
120121      NaN   Rural/Urban    Urban           0.0                  0.0   
120122      NaN   Rural/Urban    Urban           0.0                  0.0   
120123      NaN   Rural/Urban    Urban           0.0                  0.0   
120124      NaN   Rural/Urban    Urban       10416.0               6946.0   
120125      NaN   Rural/Urban    Urban           0.0                  0.0   

        Tot_Opioid_Clms      Tot_Clms  Opioid_Prscrbng_Rate  \
0            66138200.0  1.501140e+09                  4.41   
3             1814583.0  2.800821e+07                  6.48   
4               80311.0  1.639104e+06                  4.90   
5             1359955.0  2.569376e+07                  5.29   
6              998319.0  1.789430e+07                  5.58   
...                 ...           ...                   ...   
120121              NaN  0.000000e+00                   NaN   
120122              0.0  0.000000e+00                   NaN   
120123              0.0  0.000000e+00                   NaN   
120124         438797.0  2.743746e+07                  1.60   
120125              0.0  0.000000e+00                   NaN   

        Opioid_Prscrbng_Rate_5Y_Chg  Opioid_Prscrbng_Rate_1Y_Chg  \
0                             -1.32                        -0.27   
3                             -1.40                        -0.42   
4                             -2.01                        -0.90   
5                             -1.79                        -0.33   
6                             -1.08                        -0.32   
...                             ...                          ...   
120121                          NaN                          NaN   
120122                          NaN                          NaN   
120123                          NaN                          NaN   
120124                          NaN                        -0.67   
120125                          NaN                          NaN   

        LA_Tot_Opioid_Clms  LA_Opioid_Prscrbng_Rate  \
0                7290510.0                    11.02   
3                 138456.0                     7.63   
4                  13005.0                    16.19   
5                 184250.0                    13.55   
6                  76135.0                     7.63   
...                    ...                      ...   
120121                 NaN                      NaN   
120122                 0.0                      NaN   
120123                 NaN                      NaN   
120124             13865.0                     3.16   
120125         

In [9]:
cdc_df

State  Year  Month           Period  \
0        AK  2015  April  12 month-ending   
1        AK  2015  April  12 month-ending   
2        AK  2015  April  12 month-ending   
3        AK  2015  April  12 month-ending   
4        AK  2015  April  12 month-ending   
...     ...   ...    ...              ...   
44689    YC  2021    May  12 month-ending   
44690    YC  2021    May  12 month-ending   
44691    YC  2021    May  12 month-ending   
44692    YC  2021    May  12 month-ending   
44693    YC  2021    May  12 month-ending   

                                               Indicator        Data Value  \
0      Natural & semi-synthetic opioids, incl. methad...               NaN   
1                                        Cocaine (T40.5)               NaN   
2               Natural & semi-synthetic opioids (T40.2)               NaN   
3                         Number of Drug Overdose Deaths               126   
4          Psychostimulants with abuse potential (T43.6)               NaN   
...                                                  ...               ...   
44689         Synthetic opioids, excl. methadone (T40.4)             1,792   
44690                       Percent with drugs specified  99.2307692307692   
44691                                   Number of Deaths            62,248   
44692                                    Cocaine (T40.5)               962   
44693           Natural & semi-synthetic opioids (T40.2)               469   

      Percent Complete  Percent Pending Investigation     State Name  \
0                  100                       0.000000         Alaska   
1                  100                       0.000000         Alaska   
2                  100                       0.000000         Alaska   
3                  100                       0.000000         Alaska   
4                  100                       0.000000         Alaska   
...                ...                            ...            ...   
44689              100                       0.298805  New York City   
44690              100                       0.298805  New York City   
44691              100                       0.298805  New York City   
44692              100                       0.298805  New York City   
44693              100                       0.298805  New York City   

                                                Footnote Footnote Symbol  \
0      Numbers may differ from published reports usin...              **   
1      Numbers may differ from published reports usin...              **   
2      Numbers may differ from published reports usin...              **   
3      Numbers may differ from published reports usin...              **   
4      Numbers may differ from published reports usin...              **   
...                                                  ...             ...   
44689              Underreported due to incomplete data.               *   
44690              Underreported due to incomplete data.               *   
44691              Underreported due to incomplete data.               *   
44692              Underreported due to incomplete data.               *   
44693              Underreported due to incomplete data.               *   

      Predicted Value  
0                 NaN  
1                 NaN  
2                 NaN  
3                 126  
4                 NaN  
...               ...  
44689           1,864  
44690             NaN  
44691             NaN  
44692           1,005  
44693             485  

[44694 rows x 12 columns]

In [10]:
# Dropping years 2020 & 2021 and filtering indicator column 
cdc_year = cdc_df.loc[(cdc_df["Year"] != 2020) & (cdc_df["Year"] != 2021)]

cdc_indicator = cdc_year.loc[(cdc_year["Indicator"]=="Number of Drug Overdose Deaths") | (cdc_year["Indicator"]=="Number of Deaths") | (cdc_year["Indicator"]=="Natural & semi-synthetic opioids (T40.2)") | (cdc_year["Indicator"]=="Synthetic opioids, excl. methadone (T40.4)")]

In [11]:
# Changing column names for merging later
cdc_indicator = cdc_indicator.rename(columns={"State" : "State ID", "State Name" : "State"})

cdc_indicator

State ID  Year      Month           Period  \
2           AK  2015      April  12 month-ending   
3           AK  2015      April  12 month-ending   
6           AK  2015      April  12 month-ending   
9           AK  2015      April  12 month-ending   
15          AK  2015     August  12 month-ending   
...        ...   ...        ...              ...   
44465       YC  2019    October  12 month-ending   
44466       YC  2019  September  12 month-ending   
44472       YC  2019  September  12 month-ending   
44473       YC  2019  September  12 month-ending   
44477       YC  2019  September  12 month-ending   

                                        Indicator Data Value Percent Complete  \
2        Natural & semi-synthetic opioids (T40.2)        NaN              100   
3                  Number of Drug Overdose Deaths        126              100   
6                                Number of Deaths      4,133              100   
9      Synthetic opioids, excl. methadone (T40.4)        NaN              100   
15     Synthetic opioids, excl. methadone (T40.4)        NaN              100   
...                                           ...        ...              ...   
44465                            Number of Deaths     54,386              100   
44466    Natural & semi-synthetic opioids (T40.2)        251              100   
44472  Synthetic opioids, excl. methadone (T40.4)        893              100   
44473              Number of Drug Overdose Deaths      1,457              100   
44477                            Number of Deaths     54,447              100   

       Percent Pending Investigation          State  \
2                           0.000000         Alaska   
3                           0.000000         Alaska   
6                           0.000000         Alaska   
9                           0.000000         Alaska   
15                          0.000000         Alaska   
...                              ...            ...   
44465                       0.058839  New York City   
44466                       0.055099  New York City   
44472                       0.055099  New York City   
44473                       0.055099  New York City   
44477                       0.055099  New York City   

                                                Footnote Footnote Symbol  \
2      Numbers may differ from published reports usin...              **   
3      Numbers may differ from published reports usin...              **   
6      Numbers may differ from published reports usin...              **   
9      Numbers may differ from published reports usin...              **   
15     Numbers may differ from published reports usin...              **   
...                                                  ...             ...   
44465  Numbers may differ from published reports usin...              **   
44466  Numbers may differ from published reports usin...              **   
44472  Numbers may differ from published reports usin...              **   
44473  Numbers may differ from published reports usin...              **   
44477  Numbers may differ from published reports usin...              **   

      Predicted Value  
2                 NaN  
3                 126  
6                 NaN  
9                 NaN  
15                NaN  
...               ...  
44465             NaN  
44466             253  
44472             898  
44473           1,460  
44477             NaN  

[11880 rows x 12 columns]

In [12]:
# Merging Medicaid & Medicare data with an Outer Join
insurance = pd.merge(care_clean, caid_clean, how = "outer")
insurance

Year Prscrbr_Geo_Lvl  Prscrbr_Geo_Cd          State  RUCA_Cd  \
0     2019        National             NaN       National      NaN   
1     2019           State             1.0        Alabama      NaN   
2     2019           State             2.0         Alaska      NaN   
3     2019           State             4.0        Arizona      NaN   
4     2019           State             5.0       Arkansas      NaN   
...    ...             ...             ...            ...      ...   
1100  2015             NaN             NaN       Virginia      NaN   
1101  2015             NaN             NaN     Washington      NaN   
1102  2015             NaN             NaN  West Virginia      NaN   
1103  2015             NaN             NaN      Wisconsin      NaN   
1104  2015             NaN             NaN        Wyoming      NaN   

     Breakout_Type Breakout  Tot_Prscrbrs  Tot_Opioid_Prscrbrs  \
0           Totals  Overall     1239380.0             803549.0   
1           Totals  Overall       15575.0               9739.0   
2           Totals  Overall        2847.0               1935.0   
3           Totals  Overall       25876.0              16578.0   
4           Totals  Overall        9700.0               7258.0   
...            ...      ...           ...                  ...   
1100           NaN      NaN           NaN                  NaN   
1101           NaN      NaN           NaN                  NaN   
1102           NaN      NaN           NaN                  NaN   
1103           NaN      NaN           NaN                  NaN   
1104           NaN      NaN           NaN                  NaN   

      Tot_Opioid_Clms  ...  Opioid_Prscrbng_Rate  Opioid_Prscrbng_Rate_5Y_Chg  \
0          66138200.0  ...                  4.41                        -1.32   
1           1814583.0  ...                  6.48                        -1.40   
2             80311.0  ...                  4.90                        -2.01   
3           1359955.0  ...                  5.29                        -1.79   
4            998319.0  ...                  5.58                        -1.08   
...               ...  ...                   ...                          ...   
1100         583116.0  ...                  5.72                          NaN   
1101        1136474.0  ...                  8.11                          NaN   
1102         520370.0  ...                  5.60                          NaN   
1103         962286.0  ...                  8.51                          NaN   
1104          39088.0  ...                  7.32                          NaN   

      Opioid_Prscrbng_Rate_1Y_Chg  LA_Tot_Opioid_Clms  \
0                           -0.27           7290510.0   
1                           -0.42            138456.0   
2                           -0.90             13005.0   
3                           -0.33            184250.0   
4                           -0.32             76135.0   
...                           ...                 ...   
1100                        -0.62             42311.0   
1101                        -0.89            106143.0   
1102                        -1.30             25575.0   
1103                        -0.35            146248.0   
1104                        -0.48              3792.0   

      LA_Opioid_Prscrbng_Rate  LA_Opioid_Prscrbng_Rate_5Y_Chg  \
0                       11.02                           -1.73   
1                        7.63                           -2.13   
2                       16.19                           -3.73   
3                       13.55                           -2.58   
4                        7.63                           -1.69   
...                       ...                             ...   
1100                     7.26                             NaN   
1101                     9.34                             NaN   
1102                     4.91                             NaN   
1103                    15.20                             NaN   

In [15]:
# Merging CDC data & Insurace Data w/ an outer join and on Year and State
data = pd.merge(cdc_indicator, insurance, how = "outer", on = ["Year", "State"])

data

State ID  Year  Month           Period  \
0           AK  2015  April  12 month-ending   
1           AK  2015  April  12 month-ending   
2           AK  2015  April  12 month-ending   
3           AK  2015  April  12 month-ending   
4           AK  2015  April  12 month-ending   
...        ...   ...    ...              ...   
46160      NaN  2015    NaN              NaN   
46161      NaN  2015    NaN              NaN   
46162      NaN  2015    NaN              NaN   
46163      NaN  2015    NaN              NaN   
46164      NaN  2015    NaN              NaN   

                                      Indicator Data Value Percent Complete  \
0      Natural & semi-synthetic opioids (T40.2)        NaN              100   
1      Natural & semi-synthetic opioids (T40.2)        NaN              100   
2      Natural & semi-synthetic opioids (T40.2)        NaN              100   
3      Natural & semi-synthetic opioids (T40.2)        NaN              100   
4                Number of Drug Overdose Deaths        126              100   
...                                         ...        ...              ...   
46160                                       NaN        NaN              NaN   
46161                                       NaN        NaN              NaN   
46162                                       NaN        NaN              NaN   
46163                                       NaN        NaN              NaN   
46164                                       NaN        NaN              NaN   

       Percent Pending Investigation           State  \
0                                0.0          Alaska   
1                                0.0          Alaska   
2                                0.0          Alaska   
3                                0.0          Alaska   
4                                0.0          Alaska   
...                              ...             ...   
46160                            NaN     Puerto Rico   
46161                            NaN     Puerto Rico   
46162                            NaN  Virgin Islands   
46163                            NaN  Virgin Islands   
46164                            NaN  Virgin Islands   

                                                Footnote  ...  \
0      Numbers may differ from published reports usin...  ...   
1      Numbers may differ from published reports usin...  ...   
2      Numbers may differ from published reports usin...  ...   
3      Numbers may differ from published reports usin...  ...   
4      Numbers may differ from published reports usin...  ...   
...                                                  ...  ...   
46160                                                NaN  ...   
46161                                                NaN  ...   
46162                                                NaN  ...   
46163                                                NaN  ...   
46164                                                NaN  ...   

      Opioid_Prscrbng_Rate Opioid_Prscrbng_Rate_5Y_Chg  \
0                     6.81                         NaN   
1                     7.99                         NaN   
2                     6.13                         NaN   
3                     7.61                         NaN   
4                     6.81                         NaN   
...                    ...                         ...   
46160                 2.10                         NaN   
46161                 1.60                         NaN   
46162                 1.67                         NaN   
46163                  NaN                         NaN   
46164                  NaN                         NaN   

      Opioid_Prscrbng_Rate_1Y_Chg  LA_Tot_Opioid_Clms  \
0                           -0.10             18217.0   
1                            0.24              8974.5   
2                           -0.27             10617.0   
3                           -0.28              9048.0   
4                           -0.10           

In [5]:
cdc_indicator.groupby(['State ID', 'Year']).agg({'Data Value' : ['sum']}).reset_index()

NameError: name 'cdc_indicator' is not defined